In [ ]:
from google.colab import files
import PIL
import numpy as np
from io import BytesIO
from scipy.signal import convolve2d

In [ ]:
#helper function that creates gaussian blur filters

def make_gaussian_blur(width_of_kernel, std_dev = 1.):
  '''
  Input: width of kernel (e.g. 3, 5, 7), std deviation of the kernel (bigger means more blurry)
  '''
  #make sure odd
  assert type(width_of_kernel) == int and width_of_kernel % 2 == 1
  unnormalized_ = np.array(
      [[np.exp(- ((x_ - width_of_kernel // 2)**2\
                  + (y_ - width_of_kernel // 2)**2)/(2. * std_dev))
       for x_ in range(width_of_kernel)]
       for y_ in range(width_of_kernel)])
  return unnormalized_ / np.sum(unnormalized_)

In [ ]:
# allows you to upload images. you should br prompted to choose files, right below.
# this can also be accomplished by the GUI on the left.
uploaded = files.upload()

In [ ]:
# TODO provide the name of the file
# (can upload multiple images above...this just specifies one)

my_image_name = 'my_image.jpeg' #TODO provide name here!

#creates a PIL image
my_im = PIL.Image.open(BytesIO(uploaded[my_image_name]))
#converts to black and white
my_im = my_im.convert('L')
#converts to a numpy array
im_array = np.asarray(my_im, dtype = 'float')

In [ ]:
#displays black and white image
my_im

In [ ]:

#identity
identity_kernel = np.array(
    [[0, 0, 0],
     [0, 1, 0],
     [0, 0, 0]]
)

#should make image pretty blurry
blur_kernel = make_gaussian_blur(13, std_dev = 7.)

#some other kernels to try
sharpening_kernel = np.array(
    [[0, -1, 0],
     [-1, 5, -1],
     [0, -1, 0]]
)

emboss_kernel = np.array([
                          [-2, -1, 0],
                          [-1, 1, 1],
                          [0, 1, 2]
])

outline_kernel = np.array([
                           [-1, -1, -1],
                           [-1, 8, -1],
                           [-1, -1, -1]

])

#performs the convolution
convolved = convolve2d(im_array, emboss_kernel)

In [ ]:
# clipping the pixel values to be between 0 and 255 
# (otherwise, things wrap around...when converting back to an image, 
# it looks quite funny)
PIL.Image.fromarray(np.clip(convolved, a_min = 0, a_max = 255).astype('uint8'))

Some notes on color images & additional channels: note that, as in the demo in class, we made the image black and white here. You can, of course, convolve color images! But in general, one has to specify a great deal more. If, for instance, one would like to obtain a color image from the color image, one needs to specify how each of the three channels get mapped onto each of the new channels.

This is naturally described in terms of some linear algebra. A color image is like a black-and-white image, except each pixel is specified by a 3-dimensional vector instead of a scalar. When applying a convolution in order to obtain another color image, then, one specifies, instead of a 3x3 grid of scalars, a 3x3 grid of linear transformations from 3-space to 3-space. That's a lot more to specify -- if the kernel is 3 x 3, that's 3 x 3 x 3 x 3 degrees of freedom! Hence the choice to do this in black-and-white.

For convolutional neural networks, one isn't restricted in mapping color images to color images at each layer. In general, an arbitrary number of channels can be used. So convolutions, in general, are specified by grids of linear transformations from the input channel size to the output channel size. In many cases, the channel size grows in successive layers.